In [1]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

# Ce dont on a besoin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV

In [4]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [5]:
chemin_bd = r"./bd_ignore/"

In [108]:
df_merge_recom22 = pd.read_csv(chemin_bd+"resultat2/df_merge_recom21.csv")

In [59]:
len(list(df_merge_recom22['genres_y'].unique()))

1312795

In [109]:
df_merge_recom22['homepage'] = df_merge_recom22['homepage'].fillna('None')
df_merge_recom22['poster_path'] = df_merge_recom22['poster_path'].fillna('None')
df_merge_recom22['overview'] = df_merge_recom22['overview'].fillna('None')

In [110]:
caracteristiques = ['titleType', 'startYear', 'genres_y', 'primaryName']

In [111]:
merge_filmR = df_merge_recom22[df_merge_recom22['titleType'] == 'movie']
merge_serieR = df_merge_recom22[df_merge_recom22['titleType'] == 'tvSeries']

In [100]:
X = merge_filmR[caracteristiques]

In [101]:
import pandas as pd

df = pd.DataFrame({
    'genre': [['Action', 'Thriller'], ['Drame', 'Thriller', 'Comedy']]
})

In [112]:
df_merge_recom22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312795 entries, 0 to 1312794
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   homepage        1312795 non-null  object 
 1   imdb_id         1312795 non-null  object 
 2   original_title  1312795 non-null  object 
 3   overview        1312795 non-null  object 
 4   poster_path     1312795 non-null  object 
 5   titleType       1312795 non-null  object 
 6   startYear       1312795 non-null  float64
 7   genres_y        1312795 non-null  object 
 8   primaryName     1312795 non-null  object 
dtypes: float64(1), object(8)
memory usage: 90.1+ MB


In [113]:
list(df_merge_recom22['genres_y'].unique())

['Comedy',
 'Action,Crime,Drama',
 'Short',
 'Adventure,Animation,Comedy',
 'Drama,Romance',
 'Drama',
 'Crime,Drama,Musical',
 'Drama,Fantasy,Horror',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Fantasy',
 'Action,Crime,Thriller',
 'Biography,Drama,War',
 'Drama,Mystery,Thriller',
 'Drama,Music,Romance',
 'Drama,Western',
 'Drama,Romance,Sci-Fi',
 'Drama,Thriller',
 'Mystery,Sci-Fi,Thriller',
 'Drama,Mystery,Romance',
 'Drama,Music',
 'Crime,Drama,Thriller',
 'Biography,Drama,Music',
 'Drama,Sport',
 'Crime,Drama',
 'Comedy,Sci-Fi',
 'Mystery,Thriller',
 'Action,Adventure,Drama',
 'Adventure,Drama,Horror',
 'Comedy,Short',
 'Comedy,Drama,Romance',
 'Action,Comedy,Crime',
 'Comedy,Crime',
 'Drama,Music,Mystery',
 'Drama,Romance,Thriller',
 'Adventure,Comedy,Family',
 'Action,Adventure,Animation',
 'Adventure,Animation,Family',
 'Comedy,Crime,Drama',
 'Comedy,Drama,Fantasy',
 'Drama,Mystery,Sci-Fi',
 'Comedy,Drama',
 'Crime,Thriller',
 'Drama,Horror,Sci-Fi',
 'Horror,Mystery,Thriller

In [ ]:
df_merge_recom22[df_merge_recom22['genres_y'] == '\\N']

In [33]:
import ast

In [106]:
# df_merge_recom22['genres_y'] = df_merge_recom22['genres_y'].replace('\\N', None)

In [121]:
df_merge_recom22['genres_liste'] = df_merge_recom22['genres_y'].apply(lambda x: x.split(","))

In [122]:
df_merge_recom22['genres_liste']

0                            [Comedy]
1                            [Comedy]
2                            [Comedy]
3                            [Comedy]
4                            [Comedy]
                      ...            
1312790    [Action, Adventure, Drama]
1312791    [Action, Adventure, Drama]
1312792    [Action, Adventure, Drama]
1312793    [Action, Adventure, Drama]
1312794    [Action, Adventure, Drama]
Name: genres_liste, Length: 1312795, dtype: object

In [123]:
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x

In [76]:
# transformer la colonne genre en liste
#df_merge_recom22['genres_liste'] = df_merge_recom22['genres_liste'].apply(transfo_liste)
#df_merge_recom22.sample(5)
# je force la colonne genre_liste à n'être que des listes

,homepage,imdb_id,original_title,overview,poster_path,titleType,startYear,genres_y,primaryName,genres_liste2,genres_liste3
777656,NaN,tt0317646,Herø,Paul Whittington's moving short tells the stor...,NaN,short,2002.0,Short,Fangjin Li,[Short],[Short]
264572,NaN,tt13106706,Pierde Turno,NaN,/kbdMGnv79qTKPOMZuCfcnPEyA0n.jpg,short,2020.0,"Comedy,Short",Laura Contreras,"[Comedy, Short]","[Comedy, Short]"
1210087,NaN,tt3188682,Por mis Bigotes,"Though it sounds incredible, little Ulises wit...",/sI0LBFnBMqGKuXeRJT9c97EDxas.jpg,movie,2015.0,"Comedy,Drama,Family",Fernando Becerril,"[Comedy, Drama, Family]","[Comedy, Drama, Family]"
644181,NaN,tt1763290,Tarima,An unconventional love story between a man who...,NaN,movie,2010.0,Drama,Raymond Cabral,[Drama],[Drama]
1042598,NaN,tt2139851,End of the World,"In End of the World, the fate of humanity lies...",/jZsk45d9zARmvyW4pXvIhx7izRA.jpg,tvMovie,2013.0,"Action,Adventure,Drama",Serge Houde,"[Action, Adventure, Drama]","[Action, Adventure, Drama]"


In [124]:
tous_les_genres = set()

In [126]:
for genres in df_merge_recom22['genres_liste']:
  tous_les_genres.update(genres)

tous_les_genres
#ensemble qui contient tous mes genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western',
 '\\N'}

In [128]:
for genre in tous_les_genres:
  df_merge_recom22[f'genre_{genre}'] = df_merge_recom22['genres_liste'].apply(lambda x: genre in x)

df_merge_recom22

,homepage,imdb_id,original_title,overview,poster_path,titleType,startYear,genres_y,primaryName,genres_liste,...,genre_Sport,genre_\N,genre_Western,genre_Music,genre_Comedy,genre_Action,genre_Thriller,genre_Adult,genre_Fantasy,genre_Musical
0,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Tim Roth,[Comedy],...,False,False,False,False,True,False,False,False,False,False
1,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Antonio Banderas,[Comedy],...,False,False,False,False,True,False,False,False,False,False
2,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Sammi Davis,[Comedy],...,False,False,False,False,True,False,False,False,False,False
3,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Amanda De Cadenet,[Comedy],...,False,False,False,False,True,False,False,False,False,False
4,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Valeria Golino,[Comedy],...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312790,None,tt27688034,더 문,A man on Earth discovers a man in outer space....,/1tw43TQ3fgKP6QW9uyOvclWInkd.jpg,movie,2023.0,"Action,Adventure,Drama",Park Byeong-eun,"[Action, Adventure, Drama]",...,False,False,False,False,False,True,False,False,False,False
1312791,None,tt27688034,더 문,A man on Earth discovers a man in outer space....,/1tw43TQ3fgKP6QW9uyOvclWInkd.jpg,movie,2023.0,"Action,Adventure,Drama",Choi Byung-mo,"[Action, Adventure, Drama]",...,False,False,False,False,False,True,False,False,False,False
1312792,None,tt27688034,더 문,A man on Earth discovers a man in outer space....,/1tw43TQ3fgKP6QW9uyOvclWInkd.jpg,movie,2023.0,"Action,Adventure,Drama",Jonathan Ehren Groff,"[Action, Adventure, Drama]",...,False,False,False,False,False,True,False,False,False,False
1312793,None,tt27688034,더 문,A man on Earth discovers a man in outer space....,/1tw43TQ3fgKP6QW9uyOvclWInkd.jpg,movie,2023.0,"Action,Adventure,Drama",Paul de Havilland,"[Action, Adventure, Drama]",...,False,False,False,False,False,True,False,False,False,False
